# Notebook with test code

---

## Imports

### Standard library imports

In [1]:
import yaml

import sys

from datetime import (date, datetime)

import os

import pickle

import re

### Third party libraries

In [2]:
import boto3

import pandas as pd

### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%load_ext autoreload
%autoreload 2

from src.etl.ingesta_almacenamiento import (
    
    ## Functions
    get_client,
    ingesta_inicial,
    ingesta_consecutiva,
    get_s3_resource,
    request_data_to_API,
    get_s3_credentials,
    
)


from src.utils.utils import (

    get_api_token

)


from src.utils.data_dict import (
    data_dict
)

from src.utils.params_ml import(
    models_dict
)

---

## Configuration

In [5]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

## AWS base examples

### S3

#### Initial configuration

##### Resource

##### Client

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

#### Connecting to existing bucket and viewing contents

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="ingestion/consecutive"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

#### Reading pickle from S3

In [ ]:
obj_path = [file["Key"] for file in objects if "2021-03-18" in file["Key"]][0]
obj_path

In [ ]:
response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)
response

In [ ]:
body = response["Body"].read()

---

In [ ]:
pickle.loads(pickle.loads(body))

#### Creating bucket

In [ ]:
bucket_name = "comdline-test-bucket-rob"

s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
    ACL="private"
)

#### Uploading file to bucket

In [ ]:
file_to_upload = "../../admin/test_file3_for_s3.txt"
file_name = file_to_upload.split(sep="/")[-1]

In [ ]:
# s3.upload_file(file_to_upload, bucket_name, file_name)

s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

In [ ]:
[obj for obj in s3.Bucket(bucket_name).objects.all()]

#### Downloading files from bucket

In [ ]:
path_to_download = "/Users/rp_mbp/Documents/ReposRob_RobPer/ITAMmcd/semestre_2/Arquitectura_Prod_Dat/Venv_ArqPD/repos/dpa_2021/_robdir/" + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

#### Deleting bucket files

In [ ]:
s3.Object(bucket_name, file_name).delete()

---

#### Deleting bucket

## Downloading and storing Chicago data

#### Interacting with API

In [ ]:
dataset_id = "4ijn-s7e5"

In [ ]:
token = get_api_token("../conf/local/credentials.yaml")
token

In [ ]:
client = get_client(token)
client

In [ ]:
most_rec_date = '2021-04-10'

In [ ]:
soql_query = "inspection_date >= '{}'".format(most_rec_date)
soql_query

In [ ]:
x = client.get(dataset_id, 
               limit=10,
               where=soql_query
              )

In [ ]:
x

In [ ]:
len(x)

In [ ]:
xx = pd.DataFrame(x)
xx

In [ ]:
str(xx.shape)

##### Using "dumps"

In [ ]:
xx = pickle.dumps(x)

In [ ]:
xx

In [ ]:
pickle.loads(xx)

##### Using "dump"

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"

In [ ]:
pkl_store_loc

In [ ]:
pickle.dump(x, open(pkl_store_loc, 'wb'))

In [ ]:
pickle.load(open(pkl_store_loc, "rb"))

##### Storing data in S3

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"
pkl_store_loc

In [ ]:
## Storing file locally
pickle.dump(x, open(pkl_store_loc, "wb"))

In [ ]:
## Loading file from local and saving as variable
xx = pickle.dumps(pickle.load(open(pkl_store_loc, "rb")))

In [ ]:
## Saving pickle in s3
s3.put_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl",
    Body=xx
)

##### Downloading data from s3 and unpickling

In [ ]:
res_xx = s3.get_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl"
)
res_xx

In [ ]:
body = res_xx["Body"].read()

In [ ]:
pickle.loads(body)

#### Uploading info

#### Checking aws s3 contents

In [ ]:
s3 = get_s3_resource()

In [ ]:
cont_ingest_path

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=cont_ingest_path)['Contents']
objects

In [ ]:
objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-22.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-15.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-10.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

In [ ]:
rex = str(cont_dat_prefix) + "(.*).pkl"

lx = [datetime.strptime(re.search(rex, obj["Key"]).group(1), '%Y-%m-%d') for obj in objects if cont_dat_prefix in obj["Key"]]
most_rec_date = datetime.strftime(max(lx), '%Y-%m-%d')
most_rec_date

In [ ]:
s3x = pickle.loads(s3.get_object(Bucket=bucket_name, Key=(cont_ingest_path + "consecutive-inspections-2021-02-21.pkl"))['Body'].read())

In [ ]:
dfx = pd.DataFrame(s3x)
dfx

In [ ]:
dfx["inspection_date"] = pd.to_datetime(dfx["inspection_date"])

In [ ]:
print("max date: ", max(dfx["inspection_date"]))
print("min date: ", min(dfx["inspection_date"]))

In [ ]:
dfx.groupby(pd.Grouper(key="inspection_date")).count()[["inspection_id"]]

---

## Creating local directories for temporal data

In [ ]:
base_path = "../src/pipeline/luigi/ingestion_tmp/"

### Find most recent ingestion

#### Case 1: There is previous consecutive downloads

In [ ]:
lyrs = [ydir[-4:] for ydir in os.listdir(base_path + "consecutive") if "YEAR=" in ydir]
lyrs

In [ ]:
mr_yr = max(lyrs)
mr_yr

In [ ]:
lmths = [mdir[-2:] for mdir in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr) if "MONTH=" in mdir]
lmths

In [ ]:
mr_mth = max(lmths)
mr_mth

In [ ]:
lings = [ing for ing in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr + "/" + "MONTH=" + mr_mth)]
lings

In [ ]:
regex = "consecutive_inspections_" + "(.*).pkl"

In [ ]:
min([re.search(regex, ing).group(1) for ing in lings])

In [ ]:
lx = ['.DS_Store', 'consecutive_inspections_2021-03-05.pkl', 'consecutive_inspections_2021-03-10.pkl']
lx

In [ ]:
cont_dat_prefix = "consecutive_inspections_"

In [ ]:
regex = cont_dat_prefix + "(.*).pkl"

In [ ]:
lx

In [ ]:
lings

In [ ]:
most_recent_ing = max([re.search(regex, ing).group(1) for ing in lx if ".pkl" in ing])
most_recent_ing

#### Additional notes

In [ ]:
x = date.today().strftime('%Y-%m-%d')

In [ ]:
x[5:7]

---

## Creating .csv files with metadata

### Working with previous metadata pickle

#### Transformation pickle

In [ ]:
pkl_meta = "../results/metadata/transform_metadata.pkl"

In [ ]:
dfx = pickle.load(open(pkl_meta, "rb"))
dfx

In [ ]:
mx = [dfx.index[0]]
mx

In [ ]:
[mx.append(val) for val in dfx.iloc[0, :]]

In [ ]:
mx

In [ ]:
## Overwriting csv file from a dataframe (no header)
def write_csv_from_df(df, filepath, filename):
    
    ## Extracting df contents as list
    mdata_list = [df.index[0]]
    [mdata_list.append(val) for val in df.iloc[0, :]]
    
    ## Creating and writing csv file with extracted list
    with open(filepath + filename, mode="w") as metadata_file:
        metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        metadata_writer.writerow(mdata_list)

In [ ]:
filepath = "metadata_test_dir/"

In [ ]:
filename = "transformation_metadata_2.csv"

In [ ]:
write_csv_from_df(dfx, filepath, filename)

In [ ]:
dfx

#### Model selection pickle

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="model_selection"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

In [ ]:
obj_path = [file["Key"] for file in objects if "trained" in file["Key"]][0]
obj_path

In [ ]:
response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)
response

In [ ]:
body = response["Body"].read()

In [ ]:
model = pickle.loads(body)
model

In [ ]:
str(model["best_trained_model"])

In [ ]:
model["best_trained_model"]

### Creating dummy metadata

#### Line 1

In [ ]:
meta_1 = str(datetime.now())

In [ ]:
meta_2 = str(10)

In [ ]:
meta_3 = "carnegie"

In [ ]:
meta_comp = ",".join([meta_1, meta_2, meta_3])
meta_comp

In [ ]:
import csv
with open("transformation_metadata.csv", mode="w") as metadata_file:
    metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    metadata_writer.writerow([meta_1, meta_2, meta_3])

#### Creating string based on dictionary keys for metadata

In [ ]:
" | ".join([mdl for mdl in models_dict])

## Saving dictionary of dataframes as pickle

### Creating dictionary of dataframes

In [ ]:
dfx1 = {
    "col1_df1": list(range(1, 11)),
    "col2_df1": list("htkdlekdjc"),
}
dfx1 = pd.DataFrame.from_dict(dfx1)
dfx1

In [ ]:
dfx2 = {
    "col1_df2": list(range(11, 21)),
    "col2_df2": list("jfldurytgc"),
}
dfx2 = pd.DataFrame.from_dict(dfx2)
dfx2

In [ ]:
df_dict = {
    "dfx1": dfx1,
    "dfx2": dfx2
}
df_dict

In [ ]:
df_dict['dfx2']

### Storing dictionary as pickle variable and unpickling

In [ ]:
df_dict_pkl = pickle.dumps(df_dict)

In [ ]:
df_dict_x = pickle.loads(df_dict_pkl)

In [ ]:
df_dict_x['dfx2']

### Storing dictionary as pickle in local memory and unpickling

In [ ]:
pkl_file = "../data/pickles/test_df_dict_pkl.pkl"

In [ ]:
pickle.dump(df_dict, open(pkl_file, "wb"))

In [ ]:
df_dict_x = pickle.load(open(pkl_file, "rb"))

In [ ]:
df_dict_x['dfx2']

---

## Inspecting results

### From local pickles

#### Data ingested

In [ ]:
pth = "../data/raw/Food_Inspections.csv"

In [ ]:
dfx = pd.read_csv(pth)

In [ ]:
dfx

#### Extraction dataframe

In [ ]:
pth = "../data/pickles/ingest_df.pkl"

In [ ]:
dfx = pickle.load(open(pth, "rb"))

In [ ]:
dfx

#### Transformation df

In [ ]:
pth = "../data/pickles/transformation_df.pkl"

In [ ]:
dfx = pickle.load(open(pth, "rb"))

In [ ]:
dfx

#### Feature engineering picke

In [ ]:
pth = "../data/pickles/fe_results.pkl"

In [ ]:
fe_results = pickle.load(open(pth, "rb"))

In [ ]:
fe_results["df_imp_engineered_features"]

In [ ]:
fe_results["data_labels"]

In [ ]:
fe_results["ohe_reference"]

#### Models training pickle

In [ ]:
pth = "../data/pickles/mt_results.pkl"

In [ ]:
mt_results = pickle.load(open(pth, "rb"))

In [ ]:
mt_results["training_labels"]

In [ ]:
mt_results["test_labels"]

#### Model selection pickle

In [ ]:
pth = "../data/pickles/ms_results.pkl"

In [ ]:
ms_results = pickle.load(open(pth, "rb"))

In [ ]:
ms_results["model_test_predict_labels"]

In [ ]:
ms_results["model_test_predict_scores"]

#### Prediction pickle

In [59]:
pth = "../data/pickles/pr_results.pkl"

In [60]:
pr_results = pickle.load(open(pth, "rb"))

In [61]:
pr_results

,prediction_date,model_label,score_label_0,score_label_1
ids,,,,
2453552,2021-05-21,1,0.316327,0.683673
2386633,2021-05-21,0,0.714707,0.285293
2386595,2021-05-21,1,0.147465,0.852535
2386464,2021-05-21,1,0.465445,0.534555
2386398,2021-05-21,0,0.714707,0.285293
...,...,...,...,...
229233,2021-05-21,1,0.327976,0.672024
68205,2021-05-21,1,0.420492,0.579508
88234,2021-05-21,1,0.312177,0.687823


##### Predictions metadata

In [66]:
round(pr_results["model_label"].value_counts(normalize=True)[1], 2)

0.83

In [68]:
round(pr_results["score_label_1"].mean(), 2)

0.67

In [70]:
str(sel_model)

'RandomForestClassifier(max_features=10, max_leaf_nodes=50, min_samples_leaf=10,\n                       n_estimators=5, n_jobs=-1, oob_score=True,\n                       random_state=1111)'

### From AWS pickles

#### Ingestion

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="ingestion/"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][2]
# obj_path = [file["Key"] for file in objects][2]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [ ]:
ing_results = pickle.loads(response["Body"].read())

In [ ]:
ing_results

In [ ]:
ing_results["facility-type"].value_counts()

#### Transformation

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="transformation/"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [ ]:
tra_results = pickle.loads(response["Body"].read())

In [ ]:
tra_results

In [ ]:
tra_results["zip-income-class"].value_counts()

#### Feature engineering

In [6]:
bucket = "data-product-architecture-equipo-9"
key ="feature_engineering/"

In [7]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [8]:
fe_results = pickle.loads(response["Body"].read())

In [9]:
tst_ids = fe_results["data_labels"].index

In [10]:
tst_feats = fe_results["df_imp_engineered_features"]

In [ ]:
tst_feats.toarray()

#### Trained models

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="trained_models/"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [ ]:
objects

In [ ]:
mt_results = pickle.loads(response["Body"].read())

In [ ]:
dfx = mt_results["test_labels"].to_frame()

In [ ]:
dfx.reset_index(inplace=True)

In [ ]:
dfx

#### Models selection

In [11]:
bucket = "data-product-architecture-equipo-9"
key ="model_selection/"

In [12]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][-1]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [13]:
ms_results = pickle.loads(response["Body"].read())

In [14]:
sel_model = ms_results["best_trained_model"]

In [17]:
tst_pred = sel_model.predict(tst_feats)

In [21]:
tst_pred[:10]

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1])

In [26]:
tst_proba = sel_model.predict_proba(tst_feats)

In [ ]:
tst_proba[:, 0]

In [41]:
[str(datetime.now())[:10]]

['2021-05-21', '2021-05-21']

In [52]:
dfx = pd.DataFrame.from_dict(
    {
        "ids": tst_ids,
        "prediction_date": [str(datetime.now())[:10]]*len(tst_ids),
        "model_label": tst_pred,
        "score_label_0": tst_proba[:, 0],
        "score_label_1": tst_proba[:, 1]
    }
)

dfx.set_index("ids", inplace=True)

In [53]:
dfx

,prediction_date,model_label,score_label_0,score_label_1
ids,,,,
2492331,2021-05-21,1,0.114654,0.885346
2492319,2021-05-21,0,0.977958,0.022042
2492298,2021-05-21,0,0.507487,0.492513
2492346,2021-05-21,1,0.013187,0.986813
2492308,2021-05-21,1,0.350151,0.649849
...,...,...,...,...
2497263,2021-05-21,1,0.186984,0.813016
2497282,2021-05-21,1,0.451954,0.548046
2497262,2021-05-21,1,0.081200,0.918800


In [55]:
dfx.to_csv("df_predict.csv")

In [50]:
dfx["prediction"].value_counts(normalize=True)[1]

0.759

#### Bias and fairness

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="bias_fairness/"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [ ]:
bf_results = pickle.loads(response["Body"].read())

In [ ]:
bf_results

In [ ]:
bf_results.drop("score", axis=1)

In [ ]:
str(bf_results.loc[3, "score"])

In [ ]:
list(bf_results["reference_group"])[0]

#### Prediction

In [77]:
bucket = "data-product-architecture-equipo-9"
key ="predictions/"

In [78]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
obj_path = [file["Key"] for file in objects][0]

response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)

In [79]:
pr_results = pickle.loads(response["Body"].read())

In [80]:
pr_results

,prediction_date,model_label,score_label_0,score_label_1
ids,,,,
2492308,2021-05-21,1,0.296702,0.703298
2492321,2021-05-21,1,0.079945,0.920055
2492287,2021-05-21,1,0.372583,0.627417
2492306,2021-05-21,0,0.678269,0.321731
2492293,2021-05-21,1,0.476950,0.523050
...,...,...,...,...
2497291,2021-05-21,1,0.072279,0.927721
2497301,2021-05-21,0,0.562270,0.437730
2497254,2021-05-21,1,0.132779,0.867221


In [ ]:
bf_results.drop("score", axis=1)

In [ ]:
str(bf_results.loc[3, "score"])

In [ ]:
list(bf_results["reference_group"])[0]

---

## Building aequitas dataframe

### Creating initial dataframe with unique ID's and real test labels

In [ ]:
dfx = mt_results["test_labels"].to_frame()
dfx.rename(columns={"label": "test_real_labels"}, inplace=True)

### Adding the labels predicted by best model

In [ ]:
dfx["model_test_predict_labels"] = ms_results["model_test_predict_labels"]

### Adding zip and reference group

In [ ]:
rc = ["zip", "zip-income-class"]

In [ ]:
pth = "../data/pickles/transformation_df.pkl"

In [ ]:
dfxx = pickle.load(open(pth, "rb"))

In [ ]:
dfxx = dfxx.loc[:, rc]

#### Merging with labels dataframe

In [ ]:
dfx = dfx.join(dfxx, how="inner")

---

## *Notes*

---
---